# ChatBot Using PyTorch

<img src='https://github.com/taruntiwarihp/raw_images/blob/master/download.png?raw=true'>

A chatbot is a software application used to conduct an on-line chat conversation via text or text-to-speech, in lieu of providing direct contact with a live human agent. A chatbot is a type of software that can automate conversations and interact with people through messaging platforms.

In [31]:

import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [32]:
import torch
import torch.nn as nn

import nltk
import numpy as np
import pandas as pd
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.stem.porter import PorterStemmer

import json,urllib
from torch.utils.data import Dataset, DataLoader

import random
import re # Import re for entity extraction
import time
import datetime
!pip install ipywidgets

import ipywidgets as widgets
from IPython.display import display, clear_output
import json


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [33]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax
        return out


In [34]:
stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
    """
    sentence = ["hello, "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bag =   [0,     1,       0,   1,    0,      0,       0 ]
    """
    tokenized_sentence = [stem(w) for w in tokenized_sentence]

    bag = np.zeros(len(all_words), dtype = np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    return bag


In [35]:
from google.colab import drive
drive.mount('/content/drive')

url = '/content/drive/MyDrive/chatbot_nlp/intents.json'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import nltk

import json, urllib
with open(url, 'r') as f:
    intents = json.load(f)


all_words = []
tags = []
xy = []
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ['?','!','.',',']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags  = sorted(set(tags))

X_train  = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)

    label = tags.index(tag)
    y_train.append(label) # CrossEntropyLoss

X_train = np.array(X_train)
y_train = np.array(y_train)

class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # dataset[idx]
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

# Hyper paramters
batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(X_train[0])
learning_rate = 0.001
num_epochs = 1000


dataset = ChatDataset()
train_loader = DataLoader(dataset = dataset, batch_size = batch_size,
                        shuffle =True, num_workers=2)

device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)

        # Forward
        outputs = model(words)
        loss = criterion(outputs, labels)

        # backward and optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch +1) % 100 == 0:
        # f'epoch {epoch+1}/{num_epochs}, loss={loss.item():.4f}'
        print("epoch {}/{}, loss={:.4f}.".format(epoch+1,num_epochs,loss.item()))

# print(f'Final loss, loss={loss.item():.4f}')
print("Final Loss, loss{:.4f}".format(loss.item()))

epoch 100/1000, loss=0.8549.
epoch 200/1000, loss=0.0528.
epoch 300/1000, loss=0.0704.
epoch 400/1000, loss=0.0079.
epoch 500/1000, loss=0.0018.
epoch 600/1000, loss=0.0015.
epoch 700/1000, loss=0.0029.
epoch 800/1000, loss=0.0006.
epoch 900/1000, loss=0.0001.
epoch 1000/1000, loss=0.0003.
Final Loss, loss0.0003


In [37]:
data = {
    "model_state":model.state_dict(),
    "input_size":input_size,
    "output_size":output_size,
    "hiddent_size": hidden_size,
    "all_words":all_words,
    "tags": tags
}
FILE = "/content/drive/MyDrive/chatbot_nlp/data.pth"
torch.save(data, FILE)

print("Traning complete. file saved to",FILE)

Traning complete. file saved to /content/drive/MyDrive/chatbot_nlp/data.pth


In [38]:
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

with open(url,'r') as f:
  intents = json.load(f)


FILE = "/content/drive/MyDrive/chatbot_nlp/data.pth"
data = torch.load(FILE)

input_size = data['input_size']
hidden_size = data['hiddent_size']
output_size = data['output_size']
all_words = data['all_words']
tags = data['tags']
model_state = data['model_state']

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

cpu


NeuralNet(
  (l1): Linear(in_features=80, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=12, bias=True)
  (relu): ReLU()
)

In [39]:
df = pd.read_csv("/content/drive/MyDrive/chatbot_nlp/all_india_PO_list_without_APS_offices_ver2_lat_long.csv")

In [40]:
df.columns

Index(['officename', 'pincode', 'officeType', 'Deliverystatus', 'divisionname',
       'regionname', 'circlename', 'Taluk', 'Districtname', 'statename',
       'Telephone', 'Related Suboffice', 'Related Headoffice', 'longitude',
       'latitude'],
      dtype='object')

In [41]:
df = df[['officename','pincode']]
df.head()

,officename,pincode
0,Achalapur B.O,504273
1,Ada B.O,504293
2,Adegaon B.O,504307
3,Adilabad Collectorate S.O,504001
4,Adilabad H.O,504001


In [42]:
def callme():
  po = {v: k for v, k in enumerate(pf)}
  print("Debjit: Select your near Post Office \n ")

  for i,j in po.items():
    print(i,j)

  while True:
    try:
      sel = int(input("\n Enter Number"))
      if sel in po:
        break
      else:
        print("Invalid number. Please select a number from the list.")
    except ValueError:
      print("Invalid input. Please enter a number.")

  print("\n Debjit: You selected {} Post office. \n".format(po[sel]))

  sen = input("Debjit: Enter another query \n You: ")
  return sen

In [43]:


url = '/content/drive/MyDrive/chatbot_nlp/intents.json'

import json
from nltk.stem.porter import PorterStemmer
import numpy as np

stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
    """
    sentence = ["hello, "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bag =   [0,     1,       0,   1,    0,      0,       0 ]
    """
    tokenized_sentence = [stem(w) for w in tokenized_sentence]

    bag = np.zeros(len(all_words), dtype = np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    return bag

with open(url, 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ['?','!','.',',']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags  = sorted(set(tags))

X_train  = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)

    label = tags.index(tag)
    y_train.append(label) # CrossEntropyLoss

X_train = np.array(X_train)
y_train = np.array(y_train)

In [44]:
# Load intents and prepare data, including helper functions
import json
import nltk
from nltk.stem.porter import PorterStemmer
import numpy as np

stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
    """
    sentence = ["hello, "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bag =   [0,     1,       0,   1,    0,      0,       0 ]
    """
    tokenized_sentence = [stem(w) for w in tokenized_sentence]

    bag = np.zeros(len(all_words), dtype = np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    return bag

url = '/content/drive/MyDrive/chatbot_nlp/intents.json'

with open(url, 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ['?','!','.',',']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags  = sorted(set(tags))

X_train  = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)

    label = tags.index(tag)
    y_train.append(label) # CrossEntropyLoss

X_train = np.array(X_train)
y_train = np.array(y_train)

In [45]:
import json

url = '/content/drive/MyDrive/chatbot_nlp/intents.json'

with open(url, 'r') as f:
    intents = json.load(f)

# Brainstorming additional patterns for existing intents:

# greetings
intents['intents'][0]['patterns'].extend([
    "hey there",
    "good morning",
    "good afternoon",
    "good evening",
    "how are you doing"
])

# goodbye
intents['intents'][1]['patterns'].extend([
    "bye bye",
    "see ya",
    "talk to you later",
    "have a good day"
])

# thanks
intents['intents'][2]['patterns'].extend([
    "thank you very much",
    "thanks a lot",
    "appreciate it"
])

# items
intents['intents'][5]['patterns'].extend([
    "what products do you sell",
    "what items are available",
    "show me your catalog",
    "what can I buy"
])

# hours
intents['intents'][6]['patterns'].extend([
    "what are your business hours",
    "when are you open until",
    "what time do you close"
])

# order status
intents['intents'][7]['patterns'].extend([
    "where is my order",
    "track my package",
    "what is the status of my shipment"
])

# cancel order
intents['intents'][8]['patterns'].extend([
    "I want to cancel my purchase",
    "can I stop my order"
])

# New Intent Categories and their patterns and responses:

# Shipping Information
intents['intents'].append({
    "tag": "shipping",
    "patterns": [
        "what are your shipping options",
        "how much is shipping",
        "do you offer free shipping",
        "how long does shipping take",
        "what shipping carriers do you use"
    ],
    "responses": [
        "We offer standard and express shipping options. Shipping costs vary based on your location and the weight of your order.",
        "Shipping times typically range from 3-7 business days for standard shipping.",
        "Yes, we offer free standard shipping on orders over a certain amount.",
        "We use multiple carriers including FedEx, UPS, and USPS."
    ]
})

# Payment Methods
intents['intents'].append({
    "tag": "payment",
    "patterns": [
        "what payment methods do you accept",
        "can I pay with PayPal",
        "do you take debit cards",
        "is it safe to pay online"
    ],
    "responses": [
        "We accept major credit cards (Visa, Mastercard, American Express), PayPal, and debit cards.",
        "Yes, we accept PayPal.",
        "Yes, we take debit cards.",
        "Yes, our online payment system is secure and encrypted."
    ]
})

# Returns and Refunds
intents['intents'].append({
    "tag": "returns",
    "patterns": [
        "how do I return an item",
        "what is your return policy",
        "can I get a refund",
        "how long do refunds take"
    ],
    "responses": [
        "Please visit our returns page on our website for detailed instructions on how to return an item.",
        "You can return most items within 30 days of purchase for a full refund.",
        "Yes, you can get a refund if the item meets our return policy criteria.",
        "Refunds are usually processed within 5-10 business days after we receive the returned item."
    ]
})

# Account Information
intents['intents'].append({
    "tag": "account",
    "patterns": [
        "how do I create an account",
        "I forgot my password",
        "how do I update my account information",
        "can I change my email address"
    ],
    "responses": [
        "You can create an account by clicking on the 'Sign Up' link on our website.",
        "Click on the 'Forgot Password' link on the login page to reset your password.",
        "You can update your account information by logging into your account settings.",
        "Yes, you can change your email address in your account settings."
    ]
})

# Product Availability
intents['intents'].append({
    "tag": "availability",
    "patterns": [
        "is this item in stock",
        "when will this be available again",
        "do you have this in my size",
        "is this product still available"
    ],
    "responses": [
        "Please check the product page for real-time stock information.",
        "If an item is out of stock, you can often sign up for notifications to be alerted when it's back.",
        "Our product pages list the available sizes.",
        "The product page will indicate if the product is still available."
    ]
})


print("Brainstorming and data generation complete. New patterns and intents added to the 'intents' dictionary.")

Brainstorming and data generation complete. New patterns and intents added to the 'intents' dictionary.


In [46]:
print("Example of updated intents:")
print(json.dumps(intents['intents'][0], indent=4))
print(json.dumps(intents['intents'][-1], indent=4))

Example of updated intents:
{
    "tag": "greeting",
    "patterns": [
        "Hi",
        "How are you",
        "Is anyone there?",
        "Hello",
        "Good day",
        "Whats up",
        "hey there",
        "good morning",
        "good afternoon",
        "good evening",
        "how are you doing"
    ],
    "responses": [
        "Hello!",
        "Hey :-)",
        "Hi there, what can I do for you?",
        "Hi there, how can I help?"
    ]
}
{
    "tag": "availability",
    "patterns": [
        "is this item in stock",
        "when will this be available again",
        "do you have this in my size",
        "is this product still available"
    ],
    "responses": [
        "Please check the product page for real-time stock information.",
        "If an item is out of stock, you can often sign up for notifications to be alerted when it's back.",
        "Our product pages list the available sizes.",
        "The product page will indicate if the product is still

In [47]:
updated_intents_file = '/content/drive/MyDrive/chatbot_nlp/intents_updated.json'

with open(updated_intents_file, 'w') as f:
    json.dump(intents, f, indent=4)

print(f"Updated intents data saved to {updated_intents_file}")
print("The 'intents' dictionary now contains the expanded dataset.")

Updated intents data saved to /content/drive/MyDrive/chatbot_nlp/intents_updated.json
The 'intents' dictionary now contains the expanded dataset.


In [48]:
url = '/content/drive/MyDrive/chatbot_nlp/intents_updated.json'

with open(url, 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ['?','!','.',',']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags  = sorted(set(tags))

X_train  = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)

    label = tags.index(tag)
    y_train.append(label) # CrossEntropyLoss

X_train = np.array(X_train)
y_train = np.array(y_train)

print("Data loaded and processed for training.")
print(f"Number of training samples: {len(X_train)}")
print(f"Number of unique words: {len(all_words)}")
print(f"Number of unique tags: {len(tags)}")

Data loaded and processed for training.
Number of training samples: 90
Number of unique words: 134
Number of unique tags: 17


In [49]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Define the AdvancedChatbotModel class again as it's needed for loading the state dict
class AdvancedChatbotModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, num_classes):
        super(AdvancedChatbotModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        embedded = self.embedding(x)
        out, _ = self.lstm(embedded)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

# Define the helper functions again
import nltk
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

# Need to re-load intents and process data to ensure 'all_words', 'tags', and 'xy' are correct
url = '/content/drive/MyDrive/chatbot_nlp/intents_updated.json'

with open(url, 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ['?','!','.',',']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags  = sorted(set(tags))


# Create word_to_index mapping and pad data for sequence model
word_to_index = {word: i + 1 for i, word in enumerate(all_words)} # Shift indices by 1
word_to_index['<pad>'] = 0 # Assign 0 to padding
all_words_padded = ['<pad>'] + all_words # Update all_words list for saving

max_sequence_length = max(len(tokenize(p)) for intent in intents['intents'] for p in intent['patterns'])

X_train_indices = []
y_train = []
for (pattern_sentence, tag) in xy:
     # Convert tokenized sentence to indices using the updated word_to_index mapping
     indices = [word_to_index.get(stem(w), word_to_index['<pad>']) for w in pattern_sentence]

     # Pad with the padding index
     padded_indices = indices + [word_to_index['<pad>']] * (max_sequence_length - len(indices))
     X_train_indices.append(padded_indices)

     label = tags.index(tag)
     y_train.append(label) # CrossEntropyLoss

X_train_indices = np.array(X_train_indices)
y_train = np.array(y_train)

class ChatDatasetWithSequences(Dataset):
    def __init__(self, X_indices, y_data):
        self.n_samples = len(X_indices)
        self.x_data = torch.tensor(X_indices, dtype=torch.long) # Use long for indices
        self.y_data = torch.tensor(y_data, dtype=torch.long)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

# Hyperparameters for the new model
batch_size = 8 # Assuming batch_size is defined
learning_rate = 0.001 # Assuming learning_rate is defined
num_epochs = 1000 # Assuming num_epochs is defined

vocab_size = len(all_words_padded) # Including padding index
embedding_dim = 50 # Choose an appropriate embedding dimension
hidden_size = 128 # Increase hidden size
num_layers = 2 # Add multiple layers for LSTM/GRU
output_size = len(tags) # Number of intents


# Re-instantiate DataLoader with the new dataset
dataset_sequences = ChatDatasetWithSequences(X_train_indices, y_train)
train_loader_sequences = DataLoader(dataset = dataset_sequences, batch_size = batch_size,
                                    shuffle =True, num_workers=2)

# Instantiate the new model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AdvancedChatbotModel(vocab_size, embedding_dim, hidden_size, num_layers, output_size).to(device)

# Use the same loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# Retrain the model
print("\nStarting training with the new model architecture...")
for epoch in range(num_epochs):
    for (words_indices, labels) in train_loader_sequences:
        words_indices = words_indices.to(device)
        labels = labels.to(device)

        # Forward
        outputs = model(words_indices)
        loss = criterion(outputs, labels)

        # backward and optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch +1) % 100 == 0:
        print("epoch {}/{}, loss={:.4f}.".format(epoch+1,num_epochs,loss.item()))

print("Training complete with the new model.")

# Update the data dictionary for saving the new model
data = {
    "model_state": model.state_dict(),
    "vocab_size": vocab_size, # Input size is now vocab size for embedding layer
    "embedding_dim": embedding_dim,
    "hidden_size": hidden_size, # Corrected typo
    "num_layers": num_layers,
    "output_size": output_size,
    "all_words": all_words_padded, # Save updated all_words with padding
    "tags": tags,
    "max_sequence_length": max_sequence_length, # Save max sequence length for inference
    "word_to_index": word_to_index # Save word_to_index mapping
}
FILE = "/content/drive/MyDrive/chatbot_nlp/data_advanced.pth" # Save to a new file
torch.save(data, FILE)

print("Training complete. Advanced model file saved to", FILE)


Starting training with the new model architecture...
epoch 100/1000, loss=0.0036.
epoch 200/1000, loss=0.0004.
epoch 300/1000, loss=0.0002.
epoch 400/1000, loss=0.0002.
epoch 500/1000, loss=0.0001.
epoch 600/1000, loss=0.0001.
epoch 700/1000, loss=0.0000.
epoch 800/1000, loss=0.0000.
epoch 900/1000, loss=0.0000.
epoch 1000/1000, loss=0.4139.
Training complete with the new model.
Training complete. Advanced model file saved to /content/drive/MyDrive/chatbot_nlp/data_advanced.pth


In [50]:
import re

def extract_entities(query):
    """
    Extracts key entities (pincode, potential product names) from a user query.

    Args:
        query: The user's input query string.

    Returns:
        A dictionary containing the extracted entities.
    """
    entities = {}

    # Extract pincode: sequences of 6 digits
    pincode_match = re.search(r'\b(\d{6})\b', query)
    if pincode_match:
        entities['pincode'] = pincode_match.group(1)

    potential_products = []
    words = query.split()
    for i, word in enumerate(words):
        # Simple rule: capitalized word, not at the beginning of the sentence unless it's the only word
        if word[0].isupper() and len(word) > 1 and (i > 0 or len(words) == 1):
             # Exclude common capitalized words that are unlikely product names (e.g., "I", start of sentence words)
             if word.lower() not in ['i', 'the', 'a', 'an', 'is', 'what', 'where', 'how', 'can', 'do', 'tell', 'me']:
                # Consider multi-word product names (basic)
                product_name = word
                j = i + 1
                while j < len(words) and words[j][0].isupper():
                    product_name += " " + words[j]
                    j += 1
                potential_products.append(product_name)


    # Another simple rule: look for words following "about the", "info on", etc.
    query_lower = query.lower()
    product_precursors = ["about the ", "info on the ", "details about ", "price of the "]
    for precursor in product_precursors:
        match = re.search(re.escape(precursor) + r'(\w+)', query_lower)
        if match:
            potential_products.append(match.group(1).capitalize()) # Capitalize for consistency

    # Remove duplicates and add to entities, prioritize longer matches if any overlap
    if potential_products:
         # Simple deduplication and selection (could be improved)
        entities['product'] = list(set(potential_products)) # Store as a list for now

    return entities

# Example Usage:
test_queries = [
    "What is the status of my order with pincode 700099?",
    "Tell me about the Laptop",
    "Do you have the Red Shoes?",
    "I need information on the Bluetooth Speaker",
    "What is the price of the Big Screen TV?",
    "Hello, what items do you have?",
    "My pincode is 110001.",
    "How do I return the Defective Product?",
    "Just a general query."
]

print("Testing entity extraction:")
for query in test_queries:
    extracted = extract_entities(query)
    print(f"Query: '{query}'")
    print(f"Extracted Entities: {extracted}")
    print("-" * 20)


Testing entity extraction:
Query: 'What is the status of my order with pincode 700099?'
Extracted Entities: {'pincode': '700099'}
--------------------
Query: 'Tell me about the Laptop'
Extracted Entities: {'product': ['Laptop']}
--------------------
Query: 'Do you have the Red Shoes?'
Extracted Entities: {'product': ['Red Shoes?', 'Shoes?']}
--------------------
Query: 'I need information on the Bluetooth Speaker'
Extracted Entities: {'product': ['Bluetooth Speaker', 'Speaker']}
--------------------
Query: 'What is the price of the Big Screen TV?'
Extracted Entities: {'product': ['Screen TV?', 'Big', 'TV?', 'Big Screen TV?']}
--------------------
Query: 'Hello, what items do you have?'
Extracted Entities: {}
--------------------
Query: 'My pincode is 110001.'
Extracted Entities: {'pincode': '110001'}
--------------------
Query: 'How do I return the Defective Product?'
Extracted Entities: {'product': ['Defective Product?', 'Product?']}
--------------------
Query: 'Just a general query.'

In [51]:
# Create input and output widgets
input_box = widgets.Text(placeholder='Enter your message here...')
output_area = widgets.Output()

# Function to handle button click and process input
def on_send_button_clicked(b):
    with output_area:
        clear_output() # Clear previous output
        user_message = input_box.value
        print(f"You: {user_message}")
        input_box.value = "" # Clear the input box

        if user_message.lower() == 'quit':
            print(f"{bot_name}: Goodbye!")
            # Consider stopping the interaction here in a more robust UI
            return

        # --- Chatbot Logic (Adapted from the main loop) ---
        response_text = ""

        # Call the extract_entities function on raw user input
        extracted_entities = extract_entities(user_message)

        # Check if a pincode was extracted
        pincode_handled = False
        if 'pincode' in extracted_entities:
            try:
                pincode = int(extracted_entities['pincode'])
                # Use the globally loaded pincode dataframe (assuming df is loaded)
                if 'df' in globals():
                    pf = list(df['officename'][df['pincode'] == pincode ])
                    if len(pf) == 0:
                        response_text = f"{bot_name} : Sorry to inform you, We don't deliver here :("
                    else:
                        response_text = f"{bot_name} : Here are the post offices for pincode {pincode}:\n"
                        po = {v: k for v, k in enumerate(pf)}
                        for i, j in po.items():
                            response_text += f"{i}: {j}\n"
                        response_text += "\nDebjit: Enter another query" # Suggest next step
                else:
                    response_text = f"{bot_name} : Pincode data not loaded. Please run the data loading cell."

                pincode_handled = True
            except ValueError:
                 response_text = f"{bot_name} : Invalid pincode format detected."
                 pincode_handled = True

        if not pincode_handled:

            if all(v in globals() for v in ['all_words_padded', 'word_to_index', 'max_sequence_length', 'model', 'tags', 'device']):
                tokenized_sentence = tokenize(user_message)
                sentence_indices = [word_to_index.get(stem(word), word_to_index.get('<pad>', 0)) for word in tokenized_sentence] # Use .get with default for safety
                padded_sentence_indices = sentence_indices + [word_to_index.get('<pad>', 0)] * (max_sequence_length - len(sentence_indices))
                padded_sentence_indices = padded_sentence_indices[:max_sequence_length]

                X = torch.tensor(padded_sentence_indices, dtype=torch.long).unsqueeze(0).to(device)

                outputs = model(X)
                _, predicted = torch.max(outputs, dim=1)
                tag = tags[predicted.item()]

                probs = torch.softmax(outputs, dim=1)
                prob = probs[0][predicted.item()]

                if prob.item() > 0.75: # Confidence threshold
                    for intent_data in intents["intents"]: # Ensure intents is loaded
                        if tag == intent_data["tag"]:
                            # Handle product information requests using the mock API
                            if tag == "items" and 'product' in extracted_entities and extracted_entities['product']:
                                product_name = extracted_entities['product'][0]
                                product_details = get_product_info_from_mock_api(product_name) # Ensure this function is defined

                                if product_details:
                                     response_text = f"{bot_name} : Here is the information for {product_name}: Price - {product_details['price']}, Availability - {product_details['availability']}, Features - {product_details['features']}."
                                else:
                                     response_text = f"{bot_name} : Sorry, I couldn't find information for '{product_name}'. Is there anything else I can help with regarding items?"

                            else:
                                 # If no specific entity-based response, use a random pre-defined response
                                 response_text = f"{bot_name} : {random.choice(intent_data['responses'])}"

                else:
                    # Handle out-of-scope queries
                    response_text = f"{bot_name} I do not understand...contact on WhatsApp 1234567890"
            else:
                 response_text = f"{bot_name}: Chatbot components not fully loaded. Please run the necessary setup cells."

        # --- End Chatbot Logic ---

        if response_text:
             print(response_text)


# Create a send button
send_button = widgets.Button(description="Send")

# Link the button click to the function
send_button.on_click(on_send_button_clicked)

# Display the input box, send button, and output area
print("Namste !!! We are working everywhere in Pan-india. Just drop you delivery location pincode/postal code to check delivery availability or ask us at Whatsapp Helpline 1234567890.")
display(input_box, send_button, output_area)

Namste !!! We are working everywhere in Pan-india. Just drop you delivery location pincode/postal code to check delivery availability or ask us at Whatsapp Helpline 1234567890.


Text(value='', placeholder='Enter your message here...')

Button(description='Send', style=ButtonStyle())

Output()

## Sample Queries to Test the Chatbot

You can use these sample queries to interact with the chatbot and see how it responds based on the implemented advanced NLP techniques, entity extraction, and mock API integration.

**General Interactions:**
- Hello
- How are you?
- Thanks
- Bye

**Pincode Lookup:**
- My pincode is 700099
- What post offices are in 110001?
- Check delivery for pincode 504001

**Product Information (Mock API):**
- Tell me about the Laptop
- What is the price of the keyboard?
- Is the monitor in stock?
- Do you have the webcam?

**New Intents:**
- What payment methods do you accept? (Payment)
- How much is shipping? (Shipping)
- What is your return policy? (Returns and Refunds)
- How do I create an account? (Account Information)
- Is this item in stock? (Product Availability - general)

**Out-of-Scope Query:**
- Tell me about the weather today.
- What is the capital of France?